<a href="https://colab.research.google.com/github/robson-rsp/datascience/blob/main/clustering/book_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Book recommender system
Este projeto é um sistema de recomendação de livros baseado em um título escolhido pelo usuário. Ele escolhe o nome de um livro e o sistema recomenda livros parecidos. É um sistema bastante rudimentar pois o conjunto de dados não fornece atributos muito úteis para a tarefa. Mas serve para termos uma noção de como podemos fazer um sistema de recomendação e quais métricas podemos usar.

Fonte: https://www.kaggle.com/datasets/rxsraghavagrawal/book-recommender-system

# Initial imports

In [ ]:
%pip install ipython-autotime  --upgrade

In [2]:
from google.colab import drive, files
import pandas as pd
import warnings
drive.mount('/content/drive', force_remount=True)
warnings.filterwarnings("ignore")
%load_ext autotime

Mounted at /content/drive
time: 825 µs (started: 2023-04-08 22:59:13 +00:00)


In [70]:
books   = pd.read_csv("/content/drive/MyDrive/datasets/book-recommender-system/BX-Books.csv", sep=';', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv("/content/drive/MyDrive/datasets/book-recommender-system/BX-Book-Ratings.csv", sep=';', encoding='latin-1', on_bad_lines='skip')

time: 3.39 s (started: 2023-04-09 00:23:35 +00:00)


# EDA

In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


time: 64.6 ms (started: 2023-04-08 22:59:33 +00:00)


In [6]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB
time: 436 ms (started: 2023-04-08 23:00:17 +00:00)


Com certeza não vou precisar de todos os atributos. Além disso, a técnica que usarei para criar o sistema dispensa transformações de tipos de dados.

In [7]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


time: 40.2 ms (started: 2023-04-08 23:03:33 +00:00)


In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
time: 447 ms (started: 2023-04-08 23:03:46 +00:00)


# Train/Test split
Como o conjunto de dados não possui rótulos, vou usar modelos não supervisionados de agrupamento para a tarefa. Isso faz com que a separação em conjunto de treinamento e teste seja desnecessáia.

# Feature engineering

In [26]:
from scipy.sparse      import csr_matrix
from sklearn.base      import BaseEstimator, TransformerMixin
from sklearn.compose   import ColumnTransformer
from sklearn.pipeline  import Pipeline

import numpy as np

time: 823 µs (started: 2023-04-08 23:35:38 +00:00)


Vou remover alguns atributos que não serão necessário para a tarefa e renomear aqueles que ficarem.  Este projeto tem um processo de engenharia de atributos tão simples que não há necessidade de se criar classes transformadoras ou pipelines, por isso vou usar apenas funções.

In [71]:
books   = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

books   = books.rename(columns={'ISBN':'isbn', 'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'})
ratings = ratings.rename(columns={'User-ID':'user_id', 'ISBN':'isbn', 'Book-Rating':'rating'})

print(f'books:   {list(books.columns)}')
print(f'ratings: {list(ratings.columns)}')

books:   ['isbn', 'title', 'author', 'year', 'publisher']
ratings: ['user_id', 'isbn', 'rating']
time: 90.6 ms (started: 2023-04-09 00:23:42 +00:00)


Como podemos ver acima, as colunas que serão usadas não são muito úteis em um sistema de recomendação real. Um conjunto um pouco mais completo teria, palavras chave, sinopse e tema para que pudessemos fazer recomendações melhores. Neste caso, o modelo de machine learnig vai usar o título do livro, autor, editora, ano de lançamento e média de notas para fazer as recomendações. Os passos para que cheguemos a esses atributos são vistos abaixo.

**Etapa ##:** Identifico e seleciono os usuários que tenham feito mais de 100 avaliações e o livros que tenha recebido mais de 50. Os números foram escolhidos seguindo um critério pessoal.


In [72]:
def filter_rows(dataset, col_name, min_ratings):
  valid_rows = dataset[col_name].value_counts() > min_ratings
  valid_rows_ids = valid_rows[valid_rows].index
  mask = dataset[col_name].isin(valid_rows_ids)
  return dataset[mask]


ratings = filter_rows(ratings, 'user_id', 200)
ratings = filter_rows(ratings, 'isbn', 50)

time: 647 ms (started: 2023-04-09 00:23:46 +00:00)


**Etapa ##:** Crio um novo atributo na tabela "books" contendo a avaliação média de cada livro.

In [73]:
def ratings_average(dataset, ratings):
  ratings_average = ratings.groupby('isbn')['rating'].mean()
  ratings_average = ratings_average.to_dict()
  dataset['rating'] = dataset['isbn'].map(ratings_average)
  return dataset



books = ratings_average(books, ratings)

time: 52.9 ms (started: 2023-04-09 00:23:47 +00:00)


**Etapa ##:** Seleciono apenas os livros que estão presentes na tabela "ratings".

In [74]:
mask = books['isbn'].isin(list(ratings['isbn']))
books = books[mask]

time: 32.3 ms (started: 2023-04-09 00:23:49 +00:00)


**Etapa ##:** Reinicio os indexes da tabela. Além disso, removo a coluna "isbn" da tabela pois, obviamente, ela não tem nenhum valor preditivo.

In [79]:
books = books.reset_index()
books = books.drop(['isbn', 'index'], axis=1)

time: 2.36 ms (started: 2023-04-09 00:27:03 +00:00)
